In [2]:
import pandas as pd
import gc

train_df = pd.read_csv('./train_transaction.csv')
train_idt_df = pd.read_csv('./train_identity.csv')
test_df = pd.read_csv('./test_transaction.csv')
test_idt_df = pd.read_csv('./test_identity.csv')

train_df = pd.merge(train_df, train_idt_df, on='TransactionID', how='left')
test_df = pd.merge(test_df, test_idt_df, on='TransactionID', how='left')

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
train_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

In [5]:
train_len = len(train_df)
test_len = len(test_df)

i_cols = ['card1','card2','card3','card5',
          'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14',
          'D1','D2','D3','D4','D5','D6','D7','D8','D9',
          'addr1','addr2',
          'dist1','dist2'
         ]

for col in i_cols:
    temp_df = pd.concat([train_df[[col]], test_df[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()       
    train_df[col + '_fq_enc'] = train_df[col].map(fq_encode)
    test_df[col + '_fq_enc']  = test_df[col].map(fq_encode)

In [6]:
i_cols = ['card1','card2','card3','card5']

for col in i_cols:
    for agg_type in ['mean','std']:
        new_col_name = col + '_TransactionAmt_' + agg_type
        temp_df = pd.concat([train_df[[col, 'TransactionAmt']], test_df[[col,'TransactionAmt']]])
        temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                columns={agg_type: new_col_name})
        
        temp_df.index = list(temp_df[col])
        temp_df = temp_df[new_col_name].to_dict()   
    
        train_df[new_col_name] = train_df[col].map(temp_df)
        test_df[new_col_name]  = test_df[col].map(temp_df)    

all_df = train_df.append(test_df).reset_index()

del train_df, test_df
gc.collect()

174

In [7]:
d_info = all_df['DeviceInfo'].values
for i in range(len(d_info)):
    d_info[i] = str(d_info[i])
    if 'SM-' in d_info[i]:
        d_info[i] = 'SM'
    elif 'LG-' in d_info[i]:
        d_info[i] = 'LG'
    elif 'Moto' in d_info[i]:
        d_info[i] = 'Moto'
    elif 'Hisense' in d_info[i]:
        d_info[i] = 'Hisense'
    elif 'GT' in d_info[i]:
        d_info[i] = 'GT'
    elif 'Lenovo' in d_info[i]:
        d_info[i] = 'Lenovo'
    elif 'Nexus' in d_info[i]:
        d_info[i] = 'Nexus'
    elif 'HUAWEI' in d_info[i]:
        d_info[i] = 'HUAWEI'
    elif 'HTC' in d_info[i]:
        d_info[i] = 'HTC'

In [8]:
id_30 = all_df['id_30'].values
for i in range(len(id_30)):
    id_30[i] = str(id_30[i])
    if 'Android' in id_30[i]:
        id_30[i] = 'Android'
    elif 'iOS' in id_30[i]:
        id_30[i] = 'iOS'
    elif 'Mac' in id_30[i]:
        id_30[i] = 'Mac'
    elif 'Windows' in id_30[i]:
        id_30[i] = 'Windows'

In [9]:
id_31 = all_df['id_31'].values
for i in range(len(id_31)):
    id_31[i] = str(id_31[i])
    if 'chrome ' in id_31[i]:
        id_31[i] = 'chrome'
    elif 'safari' in id_31[i]:
        id_31[i] = 'safari'
    elif 'firefox' in id_31[i]:
        id_31[i] = 'firefox'
    elif 'edge' in id_31[i]:
        id_31[i] = 'edge'
    elif 'ie' in id_31[i]:
        id_31[i] = 'ie'
    elif 'samsung' in id_31[i]:
        id_31[i] = 'samsung'
    elif 'android' in id_31[i]:
        id_31[i] = 'android'

In [10]:
from sklearn.preprocessing import LabelEncoder

for col in all_df:
    if all_df[col].dtype == np.number:
        all_df[col].fillna(all_df[col].mean())

for col in list(all_df):
    if all_df[col].dtype == 'O':
        all_df[col] = all_df[col].fillna('unseen_before_label')
        all_df[col] = all_df[col].astype(str)
        le = LabelEncoder()
        le.fit(list(all_df[col]))
        all_df[col] = le.transform(all_df[col])
        all_df[col] = all_df[col].astype('category')
        
train_df = all_df[:train_len]
test_df = all_df[train_len:]

del all_df
gc.collect()

0

In [11]:
big_top_train_cols = [col for col in train_df.columns if train_df[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_test_cols = [col for col in test_df.columns if test_df[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
many_null_train_cols = [col for col in train_df.columns if train_df[col].isnull().sum() / train_df.shape[0] > 0.9]
many_null_test_cols = [col for col in test_df.columns if test_df[col].isnull().sum() / test_df.shape[0] > 0.9]

cols_to_drop = list(set(big_top_train_cols + big_top_test_cols + many_null_train_cols + many_null_test_cols))
cols_to_drop.remove('isFraud')

train_df = train_df.drop(cols_to_drop, axis=1)
test_df = test_df.drop(cols_to_drop, axis=1)

In [12]:
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold

In [13]:
folds = StratifiedKFold(n_splits=20, shuffle=True, random_state=1001)

oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['isFraud', 'TransactionID', 'index']]

In [ ]:
import optuna
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics

data  = train_df[feats]
target = train_df['isFraud']
train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)

def objective(trial):
    global train_x, test_x, train_y, test_y
    dtrain = lgb.Dataset(train_x, label=train_y)
    dtest = lgb.Dataset(test_x, label=test_y)
    
    param = {
        'objective': 'binary',
        'metric': 'binary_error',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting', ['gbdt']),
        'max_depth': -1,
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'subsample': trial.suggest_loguniform('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.5, 1),
        'max_depth': -1,
        'tree_learner': 'serial',
        'max_bin': 255,
        'seed': 42,
        'n_jobs': -1,
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'num_iterations': 800
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'binary_error')
    gbm = lgb.train(
        param, dtrain, valid_sets=[dtest], verbose_eval=False, callbacks=[pruning_callback])

    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    return 1.0 - accuracy


In [ ]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
study.optimize(objective, n_trials=100)

print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

C:\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')
[I 2019-09-03 13:12:54,446] Finished trial#0 resulted in value: 0.03425339519761572. Current best value is 0.03425339519761572 with parameters: {'boosting': 'gbdt', 'subsample_freq': 1, 'subsample': 0.6545210955215776, 'colsample_bytree': 0.64396074541576, 'num_leaves': 646, 'learning_rate': 2.3224281139450528e-07}.
[I 2019-09-03 13:17:04,511] Finished trial#1 resulted in value: 0.01598536932299255. Current best value is 0.01598536932299255 with parameters: {'boosting': 'gbdt', 'subsample_freq': 8, 'subsample': 0.7550724191573409, 'colsample_bytree': 0.8140591783663977, 'num_leaves': 81, 